In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
!pip install flask-ngrok
!pip install flask-cors
!pip install transformers
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (0.4.1.2)


In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
def get_answer(questions, texts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    answers = []
    scores = []
    for question in questions:
        for text in texts:

            inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt").to(device)
            offset_map = tokenizer.encode_plus(question, text, add_special_tokens=True, padding=True, max_length=512, return_tensors="pt", return_offsets_mapping=True)['offset_mapping'][0]
            input_ids = inputs["input_ids"].tolist()[0]
            
            text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
            model.to(device)
            res = model(**inputs)
            model.to('cpu')
            answer_start_scores = res['start_logits']
            answer_end_scores = res['end_logits']
            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1
            score = torch.max(answer_start_scores) + torch.max(answer_end_scores)
            
            
            if answer_start >= answer_end - 1:
                continue

            if score < 0.75:
                continue
            

            start = offset_map[answer_start][0]
            end = offset_map[answer_end][1]
            # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
            answer = text[start:end]
            if len(answer) == 0:
                continue
            answers.append(answer)
            if len(answers) > 10:
                return answers
    return answers

In [5]:
from flask import Flask 
from flask import request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
from flask import jsonify
from collections import defaultdict, Counter
from heapq import heappush, heappop


In [18]:
from collections import Counter, defaultdict
class URL:
    def __init__(self):
        # self.url = url
        self.asked_questions = defaultdict(set)
        self.counter = Counter()
        self.qa_sep = "#@"
        self.sep = "%$"

    def add_qa(self, question, answer):
        answer = answer.replace('\n', self.sep)
        self.asked_questions[question].add(answer)
        self.counter[question] += 1
        print("add_qa", answer)

    def get_top_qa(self, top=3):
        qas = sorted([(q, a) for q, a in self.asked_questions.items()], key=lambda x: -self.counter[x[0]])
        print(self.counter)
        return qas[:top]

    def get_answer(self, question):
        if question in self.asked_questions:
            return self.sep.join([str(answer) for answer in self.asked_questions[question]])
        return ""

    def get_top_qa_string(self, top=3):
        print("top qa before string: ", self.get_top_qa(top))
        qa = self.qa_sep.join([q+self.sep+ self.sep.join(list(a)) for q, a in self.get_top_qa(top)])
        return qa


class Database:
    def __init__(self):
        self.urls = defaultdict(URL)

    def set_url_qa(self, url, question, answer):
        self.urls[url].add_qa(question, answer)

    def get_url(self, url):
        if url in self.urls:
            return self.urls[url]
        return None

    def get_url_top_qa(self, url, top=3):
        if url not in self.urls:
            return ""
        else:
            return self.urls[url].get_top_qa_string()

    def get_answer(self, url, question):
        url_obj = self.get_url(url)
        if url_obj is None:
            return ""
        return self.urls[url].get_answer(question)


    def remove_url(self, url):
        if url not in self.urls:
            return
        del self.urls[url]

database = Database()

In [19]:
app = Flask(__name__) 
# CORS(app)
run_with_ngrok(app)    
app.config['CORS_HEADERS'] = 'Content-Type'
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})
@app.route("/") 
def home(): 
    return "<h1>GFG is great platform to learn</h1>"

# @cross_origin(headers=['Content-Type']) # Send Access-Control-Allow-Headers
@app.route('/api/query', methods = ['POST', 'GET'])
@cross_origin(headers=['Content-Type'])
def get_query_from_react():
    sep = '%$'
    data = request.get_json()
    print(data.keys())
    print(data['data'])
    
    texts = data['data'].split(sep)
    print("texts", texts)
    question = texts[0]
    contents = [text for text in texts[1:] if len(text.split(' ')) > 20]
    print("contents", contents)
    is_answer = texts[-1] == "@answer"
    url = texts[-2]
    print("number of paragraphs: ", len(contents))
    print("Question is: ", question)
    print("Contents are: ", contents)
    print("URL is ", url)
    print("is answer", is_answer)
    answers = ""
    if not is_answer:
        old_answers = database.get_answer(url, question)
        print("old answer is: ", old_answers)
        if old_answers:
            answers = old_answers
        elif len(contents):
            answers = sep.join(get_answer([question], contents))
    else: # get answer from front-end
        answers = sep.join(texts[1:-2])
    if answers:
        database.set_url_qa(url, question, answers)

    print(answers)
    print("Answer: ", answers)
    #r = "the"
    return jsonify(answers)

@app.route('/api/init', methods = ['POST', 'GET'])
@cross_origin(headers=['Content-Type'])
def get_top_question_answer():
    sep = '%$'
    data = request.get_json()
    print(data)
    url = data['data']
    print(url)
    qa = database.get_url_top_qa(url)
    print("top qa are: ", qa)
    print(jsonify(qa))
    return jsonify(qa)


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e31e7c8d88a8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Mar/2021 04:28:32] "POST /api/init HTTP/1.1" 200 -


{'data': 'https://en.wikipedia.org/wiki/BERT_(language_model)'}
https://en.wikipedia.org/wiki/BERT_(language_model)
top qa are:  
<Response 3 bytes [200 OK]>


...

{'data': 'https://en.wikipedia.org/wiki/BERT_(language_model)'}
https://en.wikipedia.org/wiki/BERT_(language_model)
Counter({'What is BERT?': 5, 'How many encoders are in BERT?': 2})
top qa before string:  [('What is BERT?', {' (language model)%$ a Transformer-based machine learning technique for natural language processing (NLP) pre-training%$From Wikipedia, the free encyclopedia%$Jump to navigation%$Jump to search%$%$Bidirectional Encoder Representations from Transformers%$ a Transformer-based machine learning technique for natural language processing%$ a Transformer-based machine learning technique%$ Pre-training of Deep Bidirectional Transformers for Language Understanding".%$ Pre-training%$ "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding".%$ Pre-training of Deep Bidirectional Transformers%$ Deep Bidirectional Transformers for Language Understanding".'}), ('How many encoders are in BERT?', {' 24%$ 24 Encoders%$ 24 Encoders with 24%$ 24 Encoders wit

127.0.0.1 - - [16/Mar/2021 04:33:28] "POST /api/query HTTP/1.1" 200 -


dict_keys(['data'])
Are the data pre-trained?%$%$https://en.wikipedia.org/wiki/BERT_(language_model)%$@question
texts ['Are the data pre-trained?', '', 'https://en.wikipedia.org/wiki/BERT_(language_model)', '@question']
contents []
number of paragraphs:  0
Question is:  Are the data pre-trained?
Contents are:  []
URL is  https://en.wikipedia.org/wiki/BERT_(language_model)
is answer False
old answer is:  

Answer:  
dict_keys(['data'])
Are the data pre-trained?%$BERT (language model)%$From Wikipedia, the free encyclopedia%$Bidirectional Encoder Representations from Transformers (BERT) is a Transformer-based machine learning technique for natural language processing (NLP) pre-training developed by Google. BERT was created and published in 2018 by Jacob Devlin and his colleagues from Google.[1][2] As of 2019, Google has been leveraging BERT to better understand user searches.[3]%$The original English-language BERT has two models:[1] (1) the BERTBASE: 12 Encoders with 12 bidirectional self

127.0.0.1 - - [16/Mar/2021 04:33:59] "POST /api/query HTTP/1.1" 200 -


add_qa Both models are pre-trained from unlabeled data extracted from the BooksCorpus[%$Current research has focused on investigating the relationship behind BERT's output as a result of carefully chosen input sequences,[8][9] analysis of internal vector representations through probing classifiers,[10][11] and the relationships represented by attention weights.[%$pre-trained using only a plain text corpus.%$Pre-training of Deep Bidirectional Transformers for Language Understanding"
Both models are pre-trained from unlabeled data extracted from the BooksCorpus[%$Current research has focused on investigating the relationship behind BERT's output as a result of carefully chosen input sequences,[8][9] analysis of internal vector representations through probing classifiers,[10][11] and the relationships represented by attention weights.[%$pre-trained using only a plain text corpus.%$Pre-training of Deep Bidirectional Transformers for Language Understanding"
Answer:  Both models are pre-trai